In [1]:
import numpy as np

x = np.array([1, 2, 2, 3, np.nan, np.nan])
n = x.size

np.flatnonzero(x[: n - 1] < x[1:n])

array([0, 2])

In [3]:
n = 10**6
n**0.4

251.1886431509581

In [21]:
def top_3(counts):
    l = [(v, k) for k, v in counts.items()]
    return sorted(l)[-3:]


top_3({"a": 3, "b": 5, "c": 10, "d": 9})

[(5, 'b'), (9, 'd'), (10, 'c')]

In [35]:
s = "abcdefg"
k = 5


def reverse_s(s, k):
    n = len(s)

    if 2 * k <= n:
        b = s[:k]
        m = s[k:-k]
        e = s[-k:]
        return "".join([b[::-1], m, e[::-1]])
    else:
        b = s[:k]
        b = b[::-1]
        m = b[n - k :]
        e = m + s[k:]
        return b[: n - k] + e[::-1]


reverse_s(s, 4)

'dcbgfea'

In [19]:
from time import perf_counter

import numpy as np

n, d = 100_000, 100
X = np.asfortranarray(np.random.rand(n, d))
mask = np.random.rand(n) < 0.5
idx = np.flatnonzero(mask)

X.T[:, idx].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [1]:
import numpy as np

from sklearn.presortedtree.binsortdt import _preprocess_X_binning

X = np.random.rand(10_000, 1)
X = np.random.geometric(0.05, size=(1_000_000, 1))
_, counts = np.unique(X, return_counts=True)
# print((1 - np.cumsum(counts) / counts.sum()).round(3))
_, X_binned, n_bins, constant_bins = _preprocess_X_binning(X, n_bins=50)
print(constant_bins[0][X_binned[0]].mean())
# np.bincount(X_binned.ravel()), n_bins, constant_bins

0.841202


In [2]:
import numpy as np

from dataloaders import DATASET_CHARACTERISTIC, DATASET_TASK, get_dataset_numpy
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, r2_score
from sklearn.presortedtree.forest import Forest

BASE_KWARGS = dict(n_estimators=20, max_depth=10, n_jobs=4)


def to_clf(y):
    return y < np.median(y)


def print_results(algo, t, score):
    t_str = f"{t:.3f}" if t < 1 else f"{t:.1f}"
    print(f"[{algo}] fit in {t_str}s; score: {score:.2f}")


def t(name, model, X_train, y_train, X_test, y_test):
    model.fit()
    pass


def train(name, compare=True, force_task=None, **kwargs):
    kwargs = {**BASE_KWARGS, **kwargs}
    X_train, y_train, X_test, y_test = get_dataset_numpy(name)
    task = force_task or DATASET_TASK[name]
    if force_task is not None:
        if force_task == "classification" and force_task != DATASET_TASK[name]:
            y_train = to_clf(y_train)
            y_test = to_clf(y_test)

    is_reg = task == "regression"

    print(f"=== dataset: {name} | task: {task} | shape: {X_train.shape} ===")

    if is_reg:
        t = perf_counter()
        f = Forest(**kwargs).fit(X_train, y_train)
        dt = perf_counter() - t
        y_pred = f.predict(X_test)
        print_results("pre-sorted", dt, r2_score(y_test, y_pred))

        if not compare:
            return
        t = perf_counter()
        f = RandomForestRegressor(**kwargs).fit(X_train, y_train)
        dt = perf_counter() - t
        print_results("scikit-learn", dt, f.score(X_test, y_test))
    else:
        t = perf_counter()
        f = Forest(**kwargs, criterion="gini").fit(X_train, y_train)
        dt = perf_counter() - t
        y_pred = f.predict(X_test)
        print_results("pre-sorted", dt, accuracy_score(y_test, y_pred))

        if not compare:
            return
        t = perf_counter()
        f = RandomForestClassifier(**kwargs, criterion="gini", max_features=1.0).fit(
            X_train, y_train
        )
        dt = perf_counter() - t
        print_results("scikit-learn", dt, f.score(X_test, y_test))


# trigger compilation:
train("abalone", compare=False)
print()
train("abalone", compare=False, force_task="classification")

=== dataset: abalone | task: regression | shape: (3341, 8) ===
[pre-sorted] fit in 6.1s; score: 0.56

=== dataset: abalone | task: classification | shape: (3341, 8) ===
[pre-sorted] fit in 5.6s; score: 0.78


In [3]:
if False:
    for dataset in ["abalone", "letters", "higgs-sampled", "cover-type", "year-msd"]:
        train(dataset)
        print()

In [4]:
for name, (n, d) in DATASET_CHARACTERISTIC.items():
    print(name, DATASET_TASK[name], n * d // int(1e6))

abalone regression 0
airline classification 1495
airline-one-hot classification 7070
bosch classification 1146
cover-type classification 31
epsilon classification 1000
epsilon-sampled classification 14
higgs classification 308
higgs-sampled classification 14
letters classification 0
msrank ranking 164
msrank-classification classification 164
synthetic regression 100
synthetic-5k-features regression 100
synthetic-classification classification 14
year-msd regression 46


In [38]:
from threadpoolctl import threadpool_limits

from sklearn.ensemble import (
    HistGradientBoostingRegressor,
)

md = 3

hgb = HistGradientBoostingRegressor(
    max_iter=5,
    learning_rate=0.001,
    max_leaf_nodes=None,
    max_depth=md,
    min_samples_leaf=1,
    categorical_features=None,
    early_stopping=False,
    tol=0.0,
    verbose=2,
)

# X, y, X_test, y_test = get_dataset_numpy('cover-type')

duplication_level = 2

# size, d = 100_000_000, 4
size, d = 2000000, 5
n = size // d // (md or 20)
print(n)

if duplication_level == 0:
    X = np.random.rand(n, d)
elif duplication_level == 1:
    X = np.random.geometric(0.02, size=(n * 2, d))
elif duplication_level == 2:
    X = np.random.geometric(1 / 3, size=(n * 4, d))
else:
    raise ValueError()
y = np.random.rand(X.shape[0]) + X.sum(axis=1)
X = X.astype(np.float32)

133333


In [25]:
# X, y, _, _ = get_dataset_numpy('year-msd')

In [26]:
from sklearn.presortedtree.binsortdt import BinSortDecisionTree

tree = BinSortDecisionTree(max_depth=md)
args = tree.preprocess_Xy(X, y)
%time tree.fit(*args)

CPU times: user 6.69 ms, sys: 0 ns, total: 6.69 ms
Wall time: 6.54 ms


In [27]:
from sklearn.presortedtree.dt import DecisionTree

tree = DecisionTree(max_depth=md)
args = tree.preprocess_Xy(X, y)
%time tree.fit(*args)

CPU times: user 5.11 ms, sys: 1.94 ms, total: 7.05 ms
Wall time: 6.83 ms


In [28]:
tree.n_nodes_

8127

In [39]:
with threadpool_limits(1):
    hgb.fit(X, y)

Binning 0.021 GB of training data: 0.061 s
Fitting gradient boosted rounds:
[1/5] 1 tree, 8 leaves, max depth = 3, in 0.010s
[2/5] 1 tree, 8 leaves, max depth = 3, in 0.011s
[3/5] 1 tree, 8 leaves, max depth = 3, in 0.010s
[4/5] 1 tree, 8 leaves, max depth = 3, in 0.010s
[5/5] 1 tree, 8 leaves, max depth = 3, in 0.015s
Fit 5 trees in 0.125 s, (40 total leaves)
Time spent computing histograms: 0.020s
Time spent finding best splits:  0.001s
Time spent applying splits:      0.022s
Time spent predicting:           0.005s


Binning 0.226 GB of training data: 0.561 s
Fitting gradient boosted rounds:
[1/100] (522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
7 trees, 408 leaves (58 on avg), max depth = 6, in 0.716s
[2/100] (522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
7 trees, 441 leaves (63 on avg), max depth = 6, in 0.722s
[3/100] (522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
7 trees, 441 leaves (63 on avg), max depth = 6, in 0.727s
[4/100] (522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
7 trees, 441 leaves (63 on avg), max depth = 6, in 0.735s
[5/100] (522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
7 trees, 441 leaves (63 on avg), max depth = 6, in 0.743s
[6/100] (522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
(522910, 54)
7 trees, 441 leaves (63 on avg), max dep

In [36]:
hgb.score(X_test, y_test)

0.58521

In [5]:
%%memit
train('higgs-sampled', compare=True)

presort time 0.9005410450045019
presorted 8.239271325001027
0.70759
sklearn 32.68900049600052
0.70878
peak memory: 849.99 MiB, increment: 376.44 MiB


In [4]:
train("higgs-sampled")

presort time 0.7919331480006804
presorted 5.5802280610005255
0.70808
sklearn 30.85482288899948
0.70779


In [2]:
for name in ["abalone", "letters"]:
    print(name)
    train(name, force_task="regression")
    print()
    train(name, force_task="classification")
    print("-" * 60)

abalone
presort time 0.0021693109993066173
presorted 0.013532944998587482
0.5417598622065609


sklearn 0.05674817399994936
0.5566754797024823

presort time 0.0012191109999548644
presorted 0.014890521997585893
0.7894736842105263
sklearn 0.054541314999369206
0.7787081339712919
------------------------------------------------------------
letters
presort time 0.01130043599914643
presorted 0.08604779500092263
0.7042431859363498
sklearn 0.12467303999801516
0.7043161478269935

presort time 0.008582839000155218
presorted 0.06427998000071966
0.8785
sklearn 0.11488601100063534
0.888
------------------------------------------------------------


In [18]:
train("cover-type")

presort time 0.6784624820029421
presorted 7.670725104999292
0.7767718839282641
sklearn 17.020560952998494
0.7768923617087191


In [19]:
train("cover-type", force_task="regression")

presort time 0.7495246979997319
presorted 10.552567317001376
0.6031479382236706
sklearn 13.79058474300109
0.5957797358987758


In [21]:
for name, (n, d) in DATASET_CHARACTERISTIC.items():
    print(name, DATASET_TASK[name], n * d // int(1e6))

abalone regression 0
airline classification 1495
airline-one-hot classification 7070
bosch classification 1146
cover-type classification 31
epsilon classification 1000
epsilon-sampled classification 14
higgs classification 308
higgs-sampled classification 14
letters classification 0
msrank ranking 164
msrank-classification classification 164
synthetic regression 1000
synthetic-5k-features regression 500
synthetic-classification classification 14
yahoo ranking 447
yahoo-classification classification 447
year-msd regression 46


In [28]:
train("yahoo")

loading from slow format


Exception: Please download dataset from https://webscope.sandbox.yahoo.com/catalog.php?datatype=c and convert it to tsv file, 0 - Label, 1 - QueryId, 2 - ...features...

In [4]:
for name in ["synthetic-classification"]:
    print(name)
    train(name, force_task="regression")
    print()
    train(name, force_task="classification")
    print("-" * 60)

synthetic-classification
presort time 0.9477527929993812
presorted 7.041883041001711
0.8817562906923824
sklearn 36.045911052999145
0.8817491963760379

presort time 1.091656377997424
presorted 1.6219052400010696
1.0
sklearn 0.16221245799897588
1.0
------------------------------------------------------------


In [6]:
# get_dataset_numpy(name)

In [ ]:
for name, (n, d) in DATASET_CHARACTERISTIC.items():
    arrays = get_dataset_numpy(name)
    if arrays is None:
        continue
    X_train, y_train, X_test, y_test = arrays
    task = DATASET_TASK[name]
    print(name, task, (n, d))

    # --- compile ---
    if name == "abalone":
        # compile:
        Forest(**kwargs).fit(X_train, y_train)
        Forest(**kwargs, criterion="gini").fit(X_train, y_train < np.median(y_train))
    if name == "letters":
        Forest(**kwargs, criterion="gini").fit(X_train, y_train)

    is_reg = task == "regression"

    if is_reg:
        t = perf_counter()
        f = Forest(**kwargs).fit(X_train, y_train)
        print("presorted", perf_counter() - t)
        y_pred = f.predict(X_test)
        print(r2_score(y_test, y_pred))

        t = perf_counter()
        f = RandomForestRegressor(**kwargs).fit(X_train, y_train)
        print("sklearn", perf_counter() - t)
        print(f.score(X_test, y_test))
    else:
        t = perf_counter()
        f = Forest(**kwargs, criterion="gini").fit(X_train, y_train)
        print("presorted", perf_counter() - t)
        y_pred = f.predict(X_test)
        print(accuracy_score(y_test, y_pred))

        t = perf_counter()
        f = RandomForestClassifier(**kwargs, criterion="gini").fit(X_train, y_train)
        print("sklearn", perf_counter() - t)
        print(f.score(X_test, y_test))

    print("=" * 60)

(412276, 90)

In [ ]:
import numpy as np

In [15]:
np.load?

Signature:
np.load(
    file,
    mmap_mode=None,
    allow_pickle=False,
    fix_imports=True,
    encoding='ASCII',
    *,
    max_header_size=10000,
)
Docstring:
Load arrays or pickled objects from ``.npy``, ``.npz`` or pickled files.

.. warning:: Loading files that contain object arrays uses the ``pickle``
             module, which is not secure against erroneous or maliciously
             constructed data. Consider passing ``allow_pickle=False`` to
             load data that is known not to contain object arrays for the
             safer handling of untrusted sources.

Parameters
----------
file : file-like object, string, or pathlib.Path
    The file to read. File-like objects must support the
    ``seek()`` and ``read()`` methods and must always
    be opened in binary mode.  Pickled files require that the
    file-like object support the ``readline()`` method as well.
mmap_mode : {None, 'r+', 'r', 'w+', 'c'}, optional
    If not None, then memory-map the file, using the gi

array([[1.    , 0.18  , 0.135 , ..., 0.0145, 0.007 , 0.01  ],
       [1.    , 0.215 , 0.15  , ..., 0.015 , 0.009 , 0.0125],
       [2.    , 0.66  , 0.53  , ..., 0.5905, 0.212 , 0.453 ],
       ...,
       [2.    , 0.595 , 0.45  , ..., 0.463 , 0.2065, 0.2535],
       [0.    , 0.625 , 0.49  , ..., 0.477 , 0.2365, 0.3185],
       [1.    , 0.41  , 0.325 , ..., 0.1325, 0.075 , 0.101 ]],
      shape=(3341, 8))